In [2]:
import sys
import subprocess

exit_code = subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
if exit_code:
    raise Exception("Non-zero return code when installing the dependencies, please try reinstalling them manually")

In [15]:
import boto3

ami_id = "ami-039622d68c309bd08"

ec2_client = boto3.client("ec2", "us-east-1")


output = ec2_client.run_instances(
  MaxCount=1,
  MinCount=1,
  InstanceType="t3.micro",
  ImageId=ami_id,
)

instance_ids = list(map(lambda i : i.get("InstanceId"), output.get("Instances")))
print(f"Launched instances with ids: {' '.join(instance_ids)}")

jdun = ec2_client.get_waiter("instance_running")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are up and running")

Launched instances with ids: i-0c02618e58ba405c5
Instances with ids: i-0c02618e58ba405c5 are up and running


In [16]:
ec2_resource = boto3.resource("ec2", "us-east-1")
instance = ec2_resource.Instance(instance_ids[0])
print(instance.public_ip_address)

54.91.99.110


**Terminate** instances and wait until they are down:

In [17]:
ec2_client.terminate_instances(InstanceIds=instance_ids)
jdun = ec2_client.get_waiter("instance_terminated")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are terminated")

Instances with ids: i-0c02618e58ba405c5 are terminated
